<a href="https://colab.research.google.com/github/acmilannesta/Bert-embedding/blob/master/Toxic_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files
files.upload()
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [0]:
!kaggle competitions download -c jigsaw-unintended-bias-in-toxicity-classification

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
import pickle
import gc
import numpy as np
import pandas as pd
from tqdm import tqdm
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, Dropout, add, concatenate, CuDNNLSTM, CuDNNGRU
from keras.layers import Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.models import Sequential, load_model, Model
from keras.losses import binary_crossentropy
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from keras import backend as K

Using TensorFlow backend.


In [0]:
MAX_LEN = 220
BATCH_SIZE = 512
NUM_EPOCHS = 4
LSTM_UNITS = 128
DENSE_HIDDEN_UNITS = 512
NUM_MODELS = 2

In [0]:
X_train = np.load('drive/My Drive/Data/X_train.npy')
X_test = np.load('drive/My Drive/Data/X_test.npy')
embedding_matrix = np.load('drive/My Drive/Data/crawl+glove.npy')

In [8]:
train = pd.read_csv('train.csv.zip')
identity_columns = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness']
# Overall
weights = np.ones((len(train),)) / 4
# Subgroup
weights += (train[identity_columns].fillna(0).values>=0.5).sum(axis=1).astype(bool).astype(np.int) / 4
# Background Positive, Subgroup Negative
weights += (( (train['target'].values>=0.5).astype(np.int) +
   (train[identity_columns].fillna(0).values<0.5).sum(axis=1).astype(bool).astype(np.int) ) > 1 ).astype(np.int) / 4
# Background Negative, Subgroup Positive
weights += (( (train['target'].values<0.5).astype(np.int) +
   (train[identity_columns].fillna(0).values>=0.5).sum(axis=1).astype(bool).astype(np.int) ) > 1 ).astype(np.int) / 4
loss_weight = 1.0 / weights.mean()
y_train = np.vstack([(train['target'].values>=0.5).astype(np.int), weights]).T
# y_aux_train = train[['target', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat']].values
del train
gc.collect()

0

In [0]:
def custom_loss(y_true, y_pred):
    return binary_crossentropy(K.reshape(y_true[:,0],(-1,1)), y_pred) * y_true[:,1]
def build_model(embedding_matrix):

    words = Input(shape=(MAX_LEN,))
    x = Embedding(*embedding_matrix.shape, weights=[embedding_matrix], trainable=False)(words)
    x = SpatialDropout1D(0.3)(x)
    x = Bidirectional(CuDNNGRU(LSTM_UNITS, return_sequences=True))(x)
    x = Bidirectional(CuDNNGRU(LSTM_UNITS, return_sequences=True))(x)

    hidden = concatenate([GlobalMaxPooling1D()(x), GlobalAveragePooling1D()(x), ])
    hidden = add([hidden, Dense(DENSE_HIDDEN_UNITS, activation='relu')(hidden)])
    hidden = add([hidden, Dense(DENSE_HIDDEN_UNITS, activation='relu')(hidden)])
    result = Dense(1, activation='sigmoid')(hidden)
#     aux_result = Dense(num_aux_targets, activation='sigmoid')(hidden)

    model = Model(inputs=words, outputs=result)
    model.compile(loss=custom_loss, 
#                   loss_weights=[loss_weight, 1.0], 
                  optimizer='adam')

    return model
def run_model(X_train, y_train, embedding_matrix):
    checkpoint_predictions = []
    weights = []
    for model_idx in range(NUM_MODELS):
        model = build_model(embedding_matrix)
        for global_epoch in range(NUM_EPOCHS):
            model.fit(
                X_train, y_train,
                batch_size=BATCH_SIZE, epochs=1, verbose=1,
                callbacks=[LearningRateScheduler(lambda epoch: 1e-3 * (0.6 ** global_epoch))]
            )
#             with open('temporary.pickle', mode='rb') as f:
#                 X_test = pickle.load(f)  # use temporary file to reduce memory
            checkpoint_predictions.append(model.predict(X_test, batch_size=2048)[0].flatten())
#             del X_test
#             gc.collect()
            weights.append(2 ** global_epoch)
        del model
        gc.collect()

    preds = np.average(checkpoint_predictions, weights=weights, axis=0)
    return preds

In [0]:
test_pred = run_model(X_train, y_train, embedding_matrix)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/1
 131584/1804874 [=>............................] - ETA: 15:51 - loss: 0.0889